# ACDC 2022 - ERA5 Exploration Jupyter Notebook
#### M. M. Lague; Aug. 2022

Loads and plots some ERA5 data on a laptop

-----

Load a bunch of packages that we might use

In [1]:
%matplotlib inline
import sys
sys.path.append('./scripts')

# netcdf/numpy/xarray
import numpy as np
import netCDF4 as nc
import numpy.matlib
import datetime
import xarray as xr
from scipy import interpolate
from numpy import ma
from scipy import stats
import scipy.io as sio
import pickle as pickle
from sklearn import linear_model
import numpy.ma as ma
import matplotlib.patches as mpatches

import scipy as sp
import pandas as pd

import time

from copy import copy 

#from joblib import Parallel, delayed
import multiprocessing
import dask
import dask.bag as db

# Plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import ticker

from matplotlib.ticker import FormatStrFormatter

from matplotlib import gridspec

import cartopy.crs as ccrs
from cartopy.util import add_cyclic_point



# OS interaction
import os
import sys
import cftime
import datetime

import glob



from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)


# from windspharm.standard import VectorWind
# import spharm

----

## Link external harddrive
At least for me, my local jupyterlab session won't access data on an external drive unless I create a symbolic link from somewhere the session *can* access - I'm putting it in the same place as this notebook.

`ln -s /Volumes/OneTouch/ACDC/ERA5_ACDC/ /Users/mlague/Documents/ACDC/ERA5_ACDC`


----

### Load a bunch of ERA5 data from an external drive


In [2]:

#era5_path = '/Users/mlague/Documents/ACDC/ERA5_ACDC/variables/raw/'

Just to get the notebook running, we'll use the single file from github. Uncomment the above and comment out the below cells to actually load a lot of data. 

In [3]:
era5_path = './'

list what surface (or non-vertical) variables we'd like, and what vertical variables we'd like. I do this separately because the vertical ones are a lot bigger, so it is a quick thing to comment out if I don't want to load vertically resolved stuff.

The ERA5 data is pretty high resolution, so the files are ALL kind of big. Xarray and dask should "lazy" load the data we ask for - it won't actually read it into memory until we try and do some math or plotting with it. 

In [4]:
sfc_varlist = ['2m_temperature']#,'skin_temperature',]
               # 'surface_latent_heat_flux','surface_sensible_heat_flux',
               # 'surface_net_solar_radiation','surface_net_thermal_radiation',
               # 'surface_solar_radiation_downwards','surface_thermal_radiation_downwards']
vert_varlist = []#'temperature']

In [5]:

filelist = []

for var in sfc_varlist + vert_varlist:
    
    print(var)
    
    # get ALL the files in the variable's directory. You can be more fancy and only load a restricted time period. 

    # use this for real loading (uncomment the next 2 lines):
    # files_temp = sorted( filter( os.path.isfile,
    #                     glob.glob(era5_path + var + '/*.nc') ) )
    
    # temporary path to get your notebook running:
    files_temp = sorted( filter( os.path.isfile,
                        glob.glob(era5_path  + '/*.nc') ) )
    
    filelist = filelist + files_temp
    
    del files_temp

# print(len(filelist))
# print(filelist)

2m_temperature


In [6]:
ds_all = xr.open_mfdataset(filelist)


## Plot something

Like a map of the 2020 - 1959 surface temperature

Note: the coastlines won't show up if you're running offline if the coastline file for cartopy isn't downloaded somewhere already

In [7]:
var = 't2m'

mapdata = ( ds_all[var].sel(time=slice('1979-01-01', '1979-12-31')).mean('time') - 
       ds_all[var].sel(time=slice('1959-01-01', '1959-12-31')).mean('time') )

lat = mapdata.lat
lon = mapdata.lon

fig = plt.figure(figsize=(8,7))


ax = plt.axes(projection=ccrs.Robinson())
ax.coastlines(linewidth=2) 
ax.set_global()

cyclic_data, cyclic_lons = add_cyclic_point(mapdata,coord=lon)

title='1979 - 1959 annual mean surface temperature'
cb_ttl='K'
cmap=plt.cm.get_cmap('RdBu_r',32)
clim=[-2,2]

cs = plt.pcolormesh(cyclic_lons,lat,cyclic_data,transform=ccrs.PlateCarree())


plt.set_cmap(copy(cmap))

plt.title(title,fontsize=16,y=1.05)

plt.cm.ScalarMappable.set_clim(cs,vmin = clim[0],vmax = clim[1])

cbar = plt.colorbar(ax=ax,orientation='horizontal',extend='both',pad=.02, shrink=0.9)
cbar.ax.tick_params(labelsize=14) 

cbar.set_label(cb_ttl,fontsize=14)


ax.coastlines() 

plt.show()

plt.close()